In [4]:
#import libraries
!pip install kaggle
import kaggle

In [5]:
#download kaggle dataset using API
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
orders.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
#extract file from zip file
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip') 
zip_ref.extractall() # extract file to dir
zip_ref.close() # close file

In [7]:
#read data from the file and handle null values
import pandas as pd
df = pd.read_csv('orders.csv',na_values=['Not Available','unknown'])
df.head()

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [8]:
#rename columns names ..make them lower case and replace space with underscore
#Method 1
df.rename(columns = {'Order Id': 'order_id','Order Date': 'order_date'}).head()

,order_id,order_date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [9]:
#rename columns names ..make them lower case and replace space with underscore
#Method 2
#First converting all the columns to lowercase
df.columns = df.columns.str.lower()
#Replacing space with '_'
df.columns = df.columns.str.replace(' ','_')

In [10]:
#derive new columns discount
df['discount'] = df['list_price']*df['discount_percent']*.01

In [11]:
#derive new columns sale price
df['sale_price'] = df['list_price']-df['discount']

In [12]:
#derive new columns profit
df['profit'] = df['sale_price'] - df['cost_price']

In [24]:
#drop columns
df.drop(columns = ['list_price','cost_price','discount_percent'], inplace = True)


In [20]:
#Convert order_date datatype from object to date
df['order_date'] = pd.to_datetime(df['order_date'],format = "%Y-%m-%d")

In [28]:
# connction to SQL Server
import sqlalchemy as sal
engine = sal.create_engine('mssql://DESKTOP-FCKBRA0\SQLEXPRESS02/Python_Project?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn=engine.connect()

In [29]:
# Loading data to sql server
df.to_sql('df_orders', con=conn , index=False, if_exists = 'append')

38